In [1]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import Day

In [2]:
data = pd.read_excel("from_06_01-5-minute-stat.xlsx", sheet_name="Table1")
data.head()

,FROM_TIME,END_TIME,AVG_RUNNING,TRIPS_COUNT,VEHICLE_COUNT
0,2018-06-01 00:05:00,2018-06-01 00:10:00,16.22,18,186
1,2018-06-01 00:10:00,2018-06-01 00:15:00,16.65,21,180
2,2018-06-01 00:15:00,2018-06-01 00:20:00,18.88,23,197
3,2018-06-01 00:20:00,2018-06-01 00:25:00,18.97,26,203
4,2018-06-01 00:25:00,2018-06-01 00:30:00,20.68,29,212


In [3]:
def getRealTripsCount(df, n=5):
    """
    获取真实数据
    """
    trips_count = df["TRIPS_COUNT"]
    realTripsCount = []
    l = len(trips_count)
    for i in range(l):
        if l - i - 1 < n:
            realTripsCount.append(0)
        else:
            realTripsCount.append(sum(trips_count[i + 1:i + 1 + n]))
    return realTripsCount

In [4]:
data["realTripsCount"] = getRealTripsCount(data,5)

In [5]:
data["FROM_TIME"] = pd.to_datetime(data["FROM_TIME"])

In [6]:
def getEstTripsCount(df,n=5):
    trips_count = df[["FROM_TIME","realTripsCount"]]
    estTripsCount = []
    l = len(trips_count)
    for i in range(l):
        if i < n-1:
            estTripsCount.append(0)
        else:
            dates = []
            for j in range(n):
                dates.append(trips_count.iloc[i,0] - (j+1)*Day())
            trips_calc = trips_count[trips_count["FROM_TIME"].isin(dates)]
            estTripsCount.append(trips_calc["realTripsCount"].mean())
    return estTripsCount

In [7]:
data["estTripsCount"] = getEstTripsCount(data,5)
data.fillna(0, inplace=True)
data.head()

,FROM_TIME,END_TIME,AVG_RUNNING,TRIPS_COUNT,VEHICLE_COUNT,realTripsCount,estTripsCount
0,2018-06-01 00:05:00,2018-06-01 00:10:00,16.22,18,186,139,0.0
1,2018-06-01 00:10:00,2018-06-01 00:15:00,16.65,21,180,151,0.0
2,2018-06-01 00:15:00,2018-06-01 00:20:00,18.88,23,197,158,0.0
3,2018-06-01 00:20:00,2018-06-01 00:25:00,18.97,26,203,172,0.0
4,2018-06-01 00:25:00,2018-06-01 00:30:00,20.68,29,212,167,0.0
